In [1]:
#%pip install yfinance

In [2]:
import requests
import numpy as np
import pandas as pd
import yfinance as yf
from gym import Env
from gym.spaces import Discrete, Box

In [3]:
def getTickerData(ticker, period, interval):
    hist = yf.download(tickers = ticker, period=period, interval=interval)
    df = pd.DataFrame(hist)
    df = df.reset_index()
    return df

In [7]:
df = getTickerData('btc-usd', 'max', '1d')
df = df.drop(columns={'Volume', 'Adj Close'})
df = df.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Date': 'date'})
df['date'] = pd.to_datetime(df['date'])
df

[*********************100%***********************]  1 of 1 completed


,date,open,high,low,close
0,2014-09-17,465.864014,468.174011,452.421997,457.334015
1,2014-09-18,456.859985,456.859985,413.104004,424.440002
2,2014-09-19,424.102997,427.834991,384.532013,394.795990
3,2014-09-20,394.673004,423.295990,389.882996,408.903992
4,2014-09-21,408.084991,412.425995,393.181000,398.821014
...,...,...,...,...,...
3139,2023-04-22,27265.894531,27872.142578,27169.570312,27817.500000
3140,2023-04-23,27816.144531,27820.244141,27400.314453,27591.384766
3141,2023-04-24,27591.730469,27979.982422,27070.849609,27525.339844
3142,2023-04-25,27514.873047,28371.078125,27207.931641,28307.597656


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3144 non-null   datetime64[ns]
 1   open    3144 non-null   float64       
 2   high    3144 non-null   float64       
 3   low     3144 non-null   float64       
 4   close   3144 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 122.9 KB


In [ ]:
class TradingEnv(Env):
    def __init__(self) -> None:
        super().__init__()
    def step():
        pass
    def reset():
        pass
    def close():
        pass
    def render():
        pass